## Examen Scraping de données à l'aide de `Beautiful Soup`

In [133]:
import bs4
from bs4 import BeautifulSoup
import requests

Le site que l'on va scraper : https://www.oldbookillustrations.com/illustrations/

On veut récupérer la liste des illustrations présentes sur chaque page ainsi que sauvegarder le lien vers l'image et vers la page associée à cette illustration.

Dans un second temps sur la page de l'illustration on veut récupérer des informations textuelles.

### Requête HTTP

Le site a mis au point un filtre qui refuse de nous envoyer le contenu HTML à partir d'un script (code 403 : Forbidden). Heureusement en précisant des "*headers*" on peut facilement contourner ceci.

In [134]:
#Dans cette cellule, on effectue la requête vers la page à scraper afin de récupérer l'HTML.
#Afin que personne ne soit bloqué, le code est déjà écrit

url = "https://www.oldbookillustrations.com/illustrations/page/1"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
response = requests.get(url, headers=headers)
if response.ok :
  html = response.text
else :
  #Si jamais la requête est refusée, l'HTML va être récupérée d'une archive
  !wget https://raw.githubusercontent.com/giganttheo/cours/master/scraping/oldbookillustrations.html
  with open("./oldbookillustrations.html", "r") as f:
    html = f.read()

/!\ Au cas où cela ne fonctionne pas, et pour s'assurer des résultats, on peut travailler avec une sauvegarde du fichier HTML que j'ai archivée pour être sûr de tous pouvoir continuer !

### Scraper la liste des illustrations (8 points)

In [135]:
soup = BeautifulSoup(html, "html.parser")
soup

 <!DOCTYPE html>

<!--[if IE 7]>
	<html class="ie ie7" lang="en-US">
	<![endif]-->
<!--[if IE 8]>
	<html class="ie ie8" lang="en-US">
	<![endif]-->
<!--[if !(IE 7) & !(IE 8)]><!-->
<html lang="en-US">
<!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="width=device-width" name="viewport"/>
<title>Illustrations – Old Book Illustrations</title>
<link href="https://www.oldbookillustrations.com/illustrations/feed/" rel="alternate" title="Old Book Illustrations » Illustrations Feed" type="application/rss+xml"/>
<link href="https://www.oldbookillustrations.com/wp-content/plugins/obi-related-illustrations/includes/assets/css/styles.css?ver=4.9.20" id="related-posts-by-taxonomy-css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://www.oldbookillustrations.com/wp-content/plugins/wp-pagenavi/pagenavi-css.css?ver=2.70" id="wp-pagenavi-css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://www.oldbookillustrations.com/wp-content/themes/obi_theme/style.

On veut récupérer les informations de toutes les illustrations de la page sous forme d'une liste.

Chaque élément de la liste sera un dictionnaire :

```
{
  "url": *Lien vers la page de l'illustration*,
  "alt": *Titre de l'illustration*,
  "src": *Lien vers l'image (icone)*,
}
```

Créer une fonction `scrape_infos` telle que lorsqu'on lui donne la variable `soup` définie précédemment, elle renvoie la liste des dictionnaires ci-dessus, contenant l'information de chaque élément de la page.

In [136]:
# CELLULE A COMPLETER

def scrape_infos(soup):
  list_dict = []
  imgs = soup.find_all("figure") #Selectionner les blocs d'image
  for img in range(0, len(imgs)) :
    dict_img = {"url": imgs[img].a["href"],
                "src": imgs[img].img["src"],
                "alt": imgs[img].img["alt"]} #selectionner le texte qui correspond au titre de l'illustration
    list_dict.append(dict_img)
  return list_dict

In [137]:
scrape_infos(soup)

[{'alt': 'A Parisian couple walks in the street, wrapped up in warm clothes and sickly, sniffling along',
  'src': 'https://www.oldbookillustrations.com/wp-content/uploads/2022/05/flu-paris-220x220.jpg',
  'url': 'https://www.oldbookillustrations.com/illustrations/flu-paris/'},
 {'alt': 'Parisians are out at night, tilting their heads back to catch a glimpse of the comet of 1857',
  'src': 'https://www.oldbookillustrations.com/wp-content/uploads/2022/05/waiting-for-comet-220x220.jpg',
  'url': 'https://www.oldbookillustrations.com/illustrations/waiting-for-comet/'},
 {'alt': 'Three women take a solemn oath over the top hat one of them is holding',
  'src': 'https://www.oldbookillustrations.com/wp-content/uploads/2022/05/sacred-duty-220x220.jpg',
  'url': 'https://www.oldbookillustrations.com/illustrations/sacred-duty/'},
 {'alt': 'A crowd of travelers is standing on a railway platform, enduring a long wait under the rain',
  'src': 'https://www.oldbookillustrations.com/wp-content/uploa

In [138]:
#Executer cette cellule pour vérifier qu'on a bien les bons résultats

try :
  list_dict = scrape_infos(soup)
  assert len(list_dict) == 12
  assert list_dict[-1]["alt"] == "Ciampolo stands against a background of waves and flames as a devil tears at his arm with a hook"
  assert list_dict[2]["src"] == "https://www.oldbookillustrations.com/wp-content/uploads/2022/03/echinocereus-dasyacanthus-220x220.jpg"
  assert list_dict[1]["url"] == "https://www.oldbookillustrations.com/illustrations/echinocereus-berlandieri/"
  print("OK")
except :
  print("Il y a une erreur dans ton code, pense à récupérer le contenu textuel des balises avec .get_text() par exemple")

Il y a une erreur dans ton code, pense à récupérer le contenu textuel des balises avec .get_text() par exemple


### Scraper les détails d'une illustration (12 points)

Maintenant qu'on a récupéré la liste des illustrations de la page, on aimerait récupérer davantage d'informations sur la page de chaque illustration.

On va s'intéresser à une seule page dans l'exemple, mais le code doit être généralisable à n'importe quelle page d'illustration sur le site.

L'illustration dont on va tirer les informations est la suivante : https://www.oldbookillustrations.com/illustrations/echinocereus-berlandieri/

In [140]:
#Dans cette cellule, on effectue la requête vers la page à scraper afin de récupérer l'HTML.
#Afin que personne ne soit bloqué, le code est déjà écrit

url = "https://www.oldbookillustrations.com/illustrations/echinocereus-berlandieri/"
response = requests.get(url, headers=headers)
if not response.ok :
  html = response.text
else :
  #Si jamais la requête est refusée, l'HTML va être récupérée d'une archive
  !wget https://raw.githubusercontent.com/giganttheo/cours/master/scraping/echinocereus.html
  with open("./echinocereus.html", "r") as f:
    html = f.read()

--2022-05-25 15:04:52--  https://raw.githubusercontent.com/giganttheo/cours/master/scraping/echinocereus.html
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30583 (30K) [text/plain]
Saving to: ‘echinocereus.html.4’

echinocereus.html.4 100%[===================>]  29.87K  --.-KB/s    in 0s      

2022-05-25 15:04:52 (116 MB/s) - ‘echinocereus.html.4’ saved [30583/30583]



In [141]:
soup = BeautifulSoup(html, "html.parser")
soup

 <!DOCTYPE html>

<!--[if IE 7]>
	<html class="ie ie7" lang="en-US">
	<![endif]-->
<!--[if IE 8]>
	<html class="ie ie8" lang="en-US">
	<![endif]-->
<!--[if !(IE 7) & !(IE 8)]><!-->
<html lang="en-US">
<!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="width=device-width" name="viewport"/>
<title>Echinocereus berlandieri – Old Book Illustrations</title>
<link href="https://www.oldbookillustrations.com/wp-content/plugins/obi-related-illustrations/includes/assets/css/styles.css?ver=4.9.20" id="related-posts-by-taxonomy-css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://www.oldbookillustrations.com/wp-content/plugins/wp-pagenavi/pagenavi-css.css?ver=2.70" id="wp-pagenavi-css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://www.oldbookillustrations.com/wp-content/themes/obi_theme/style.css?ver=5.4.3" id="obi-style-css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://www.oldbookillustrations.com/wp-content/themes/obi_them

* 1) A l'aide de la méthode `.select` ([doc](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors)) qui recherche dans le document `HTML` avec des [sélecteurs CSS](https://www.w3schools.com/cssref/css_selectors.asp), on veut récupérer la liste des sources.

In [142]:
# CELLULE A COMPLETER

sources = soup.select("[class~=out]")[0].get_text()
sources

'Botanicus, the Internet Archive '

In [143]:
#Executer cette cellule pour vérifier qu'on a bien les bons résultats
try :
  assert type(sources) == list
except :
  print("\'sources\' devrait être une liste")
try :
  assert type(sources[0]) == str
except :
  print("Pense à récupérer le contenu textuel de chaque élément de la liste")
try :
  assert 'the Internet Archive' in sources
  print("OK")
except :
  print("Il y a surement une erreur dans ton code")

'sources' devrait être une liste
OK


* 2) A l'aide d'une expression régulière et de la bibliothèque `python` `re`, récupérer dans la page le nom de l'artiste.

*Rappel : on peut utiliser `re.compile(a)` afin de passer une expression régulière `a` en argument de la méthode `soup.find` ou `soup.find_all`*

In [144]:
import re # Bibliothèque python qui permettra d'utiliser des expressions régulières

In [145]:
# CELLULE A COMPLETER

name = soup.find(re.compile("(dd)")).text
name

'Roetter, Paulus'

In [146]:
#Executer cette cellule pour vérifier qu'on a bien les bons résultats
try :
  assert type(name) == str
except :
  print("Pense à récupérer le contenu textuel de l'élément récupéré")
try :
  assert 'Roetter' in name
  print("OK")
except :
  print("Il y a surement une erreur dans ton code")

OK


* 3) Avec une méthode au choix, on veut récupérer la liste des *keywords* associés à l'illustration

In [147]:
# CELLULE A COMPLETER

keywords = soup.select("[itemprop~=keywords]")[0].get_text() 
keywords

'1880s, 19th century, black & white, Cactaceae, flower, reference book'

In [148]:
#Executer cette cellule pour vérifier qu'on a bien les bons résultats
try :
  assert type(keywords) == list
  assert len(keywords) > 0
except :
  print("\'keywords\' devrait être une liste")
try :
  assert type(keywords[3]) == str
  print("OK")
except :
  print("Pense à récupérer le contenu textuel des éléments récupérés")

'keywords' devrait être une liste
OK


* 4) Avec une ou des méthodes au choix, on aimerait récupérer un maximum du reste des informations concernant l'illustration : le titre du livre dont elle est issue, la technique et le sujet de l'illustration et la description complète.

In [149]:
# CELLULE A COMPLETER
title = soup.select("[itemprop~=name]")[5].get_text()
title

technique = soup.select("[itemprop~=artMedium]")[0].get_text()
technique

'Metal engraving'

* 5) [BONUS] Enfin on peut sauvegarder nos résultats dans un fichier `json`
On pourra utiliser la bibliothèque `json` de `python` : [guide](https://moonbooks.org/Articles/How-to-save-a-dictionary-in-a-json-file-with-python-/)

In [150]:
# CELLULE A COMPLETER
import json

dict = {
    "keywords": keywords,
    "source" : sources,
    "name": name,
    "book_info" : {
        "book_title": title,
        "technique": technique,
    },
}

with open('ZEJLI_Oualid_Exam_Scraping.json', 'w') as fp:
    json.dump(dict, fp)